Start by defining a `Python` function that we want to compute.

In [1]:
def f(a, b, c):
    return (a & b) ^ c

Generate a circuit that computes this function. To implement the logical operations we use  standard verilog gates, which are available in `mantle.verilog.gates`.

In [2]:
import magma as m
import mantle

class VerilatorExample(m.Circuit):
    IO = ["a", m.In(m.Bit), "b", m.In(m.Bit), "c", m.In(m.Bit), "d", m.Out(m.Bit)]
    
    @classmethod
    def definition(io):
        m.wire(f(io.a, io.b, io.c), io.d)

m.compile("build/VerilatorExample", VerilatorExample, "coreir-verilog")
%cat build/VerilatorExample.v

module corebit_xor (input in0, input in1, output out);
  assign out = in0 ^ in1;
endmodule

module corebit_and (input in0, input in1, output out);
  assign out = in0 & in1;
endmodule

module VerilatorExample (input a, input b, input c, output d);
wire and_inst0_out;
wire xor_inst0_out;
corebit_and and_inst0(.in0(a), .in1(b), .out(and_inst0_out));
corebit_xor xor_inst0(.in0(and_inst0_out), .in1(c), .out(xor_inst0_out));
assign d = xor_inst0_out;
endmodule



Next, generate a verilator test harness in `C++` for the circuit. The test vectors are generated using the python function `f`. The verilator test bench compares the output of the simulator to those test vectors.

In [3]:
from fault.test_vectors import generate_function_test_vectors
from fault.verilator_target import VerilatorTarget

test_vectors = generate_function_test_vectors(VerilatorExample, f)

VerilatorTarget(VerilatorExample, test_vectors).run()
%cat build/test_VerilatorExample.cpp

TypeError: expected str, bytes or os.PathLike object, not list

In [ ]:
import fault
tester = fault.Tester(FullAdder)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)
# Note we currently have to tell magma to use the "verilog" 
# backend when working with a top-level verilog file
tester.compile_and_run("verilator", magma_output="verilog")